In [45]:
import pandas as pd
import numpy as np

df_train=pd.read_csv("/Users/imanurrahmanemon/emon/atis_train.csv")
df_test=pd.read_csv("/Users/imanurrahmanemon/emon/atis_test.csv")


In [46]:
df_train.head()

,id,intent,text,slots
0,0,flight,i want to fly from boston at 838 am and arrive...,O O O O O B-fromloc.city_name O B-depart_time....
1,1,flight,what flights are available from pittsburgh to ...,O O O O O B-fromloc.city_name O B-toloc.city_n...
2,2,flight_time,what is the arrival time in san francisco for ...,O O O B-flight_time I-flight_time O B-fromloc....
3,3,airfare,cheapest airfare from tacoma to orlando,B-cost_relative O O B-fromloc.city_name O B-to...
4,4,airfare,round trip fares from pittsburgh to philadelph...,B-round_trip I-round_trip O O B-fromloc.city_n...


In [48]:
df_train.columns = ['id','intent', 'text','slots']
df_train.head()

,id,intent,text,slots
0,0,flight,i want to fly from boston at 838 am and arrive...,O O O O O B-fromloc.city_name O B-depart_time....
1,1,flight,what flights are available from pittsburgh to ...,O O O O O B-fromloc.city_name O B-toloc.city_n...
2,2,flight_time,what is the arrival time in san francisco for ...,O O O B-flight_time I-flight_time O B-fromloc....
3,3,airfare,cheapest airfare from tacoma to orlando,B-cost_relative O O B-fromloc.city_name O B-to...
4,4,airfare,round trip fares from pittsburgh to philadelph...,B-round_trip I-round_trip O O B-fromloc.city_n...


In [47]:
df_train['intent']

0            flight
1            flight
2       flight_time
3           airfare
4           airfare
           ...     
4973        airfare
4974         flight
4975        airline
4976         flight
4977         flight
Name: intent, Length: 4978, dtype: object

In [33]:
df_train['intent'].unique()

array(['flight', 'flight_time', 'airfare', 'aircraft', 'ground_service',
       'airport', 'airline', 'distance', 'abbreviation', 'ground_fare',
       'quantity', 'city', 'flight_no', 'capacity', 'flight+airfare',
       'meal', 'restriction', 'airline+flight_no',
       'ground_service+ground_fare', 'airfare+flight_time', 'cheapest',
       'aircraft+flight+flight_no'], dtype=object)

In [36]:
from keras.preprocessing.text import Tokenizer

from keras.utils import pad_sequences
from keras.utils import to_categorical
MAX_SEQUENCE_LENGTH = 10
MAX_NUM_WORDS = 5000
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(df_train['text'])
sequences = tokenizer.texts_to_sequences(df_train['text'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(df_train['intent'])
x_train=data
y_train=labels
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 889 unique tokens.
Shape of data tensor: (4978, 10)
Shape of label tensor: (4978, 22)


In [37]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, Flatten
from keras.models import Model
from keras.models import Sequential
from keras.layers import LSTM
atis_rnnmodel = Sequential()
atis_rnnmodel.add(Embedding(MAX_NUM_WORDS, 128))
atis_rnnmodel.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
atis_rnnmodel.add(Dense((8), activation='sigmoid'))
atis_rnnmodel.compile(loss='binary_crossentropy',
 optimizer='adam',
 metrics= ['accuracy'])

In [42]:
print("There are {} rows and {} columns".format(df_train.shape[0], df_train.shape[1]))

There are 4978 rows and 4 columns


In [49]:
unique_values = df_train['intent'].unique()

print(f'Unique values in the column: {unique_values}')


Unique values in the column: ['flight' 'flight_time' 'airfare' 'aircraft' 'ground_service' 'airport'
 'airline' 'distance' 'abbreviation' 'ground_fare' 'quantity' 'city'
 'flight_no' 'capacity' 'flight+airfare' 'meal' 'restriction'
 'airline+flight_no' 'ground_service+ground_fare' 'airfare+flight_time'
 'cheapest' 'aircraft+flight+flight_no']


In [51]:
value_counts = df_train['intent'].value_counts()

print(f'Value counts:\n{value_counts}')


Value counts:
intent
flight                        3666
airfare                        423
ground_service                 255
airline                        157
abbreviation                   147
aircraft                        81
flight_time                     54
quantity                        51
flight+airfare                  21
airport                         20
distance                        20
city                            19
ground_fare                     18
capacity                        16
flight_no                       12
meal                             6
restriction                      6
airline+flight_no                2
ground_service+ground_fare       1
airfare+flight_time              1
cheapest                         1
aircraft+flight+flight_no        1
Name: count, dtype: int64


In [53]:
df = df_train[df_train["intent"].str.contains("#")==False]
df.intent.value_counts()

intent
flight                        3666
airfare                        423
ground_service                 255
airline                        157
abbreviation                   147
aircraft                        81
flight_time                     54
quantity                        51
flight+airfare                  21
airport                         20
distance                        20
city                            19
ground_fare                     18
capacity                        16
flight_no                       12
meal                             6
restriction                      6
airline+flight_no                2
ground_service+ground_fare       1
airfare+flight_time              1
cheapest                         1
aircraft+flight+flight_no        1
Name: count, dtype: int64

In [55]:
!python -m spacy download en_core_web_md

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_md-3.7.1/en_core_web_md-3.7.1-py3-none-any.whl (42.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [56]:
# import libraries and load spacy English model
import spacy
import numpy as np

nlp = spacy.load("en_core_web_md")
print("Number of vectors: {}".format(nlp.vocab.vectors_length))


Number of vectors: 300


In [65]:
training_df = pd.read_csv('/Users/imanurrahmanemon/emon/atis_train.csv')
training_df = training_df[training_df["intent"].str.contains("#") == False]

sen_train = training_df['text'].tolist()
labels_train = training_df['intent'].tolist()

# Load test dataset
test_df = pd.read_csv('/Users/imanurrahmanemon/emon/atis_test.csv')
test_df = test_df[test_df["intent"].str.contains("#") == False]
sen_test = test_df['text'].tolist()
labels_test = test_df['intent'].tolist()

In [66]:
train_size = len(labels_train)
test_size = len(labels_test)


print('Train data has {} rows and test data has {} rows'.format(train_size, test_size))

Train data has 4978 rows and test data has 893 rows


In [67]:
def encode_sentences(sentences):
    # Calculate number of sentences
    n_sentences = len(sentences)
    X = np.zeros((n_sentences, 300))

    # Iterate over the sentences
    for idx, sentence in enumerate(sentences):
        # Pass each sentence to the nlp object to create a document
        doc = nlp(sentence)
        # Save the document's .vector attribute to the corresponding row in     
        # X
        X[idx, :] = doc.vector
    return X

train_X = encode_sentences(sen_train)
test_X = encode_sentences(sen_test)

In [68]:
# encode labels i.e turn text labels into integers
from sklearn.preprocessing import LabelEncoder

# instantiate label encoder object
le = LabelEncoder()

labels_test = le.fit_transform(labels_test)
labels_train = le.fit_transform(labels_train)

In [69]:
# Import SVC
from sklearn.svm import SVC
# Train the model
clf = SVC(C=1)
clf.fit(train_X, labels_train)

SVC(C=1)

In [70]:
def validate_clf(X,y):
    # Validate model on training set
    y_pred = clf.predict(X)
    
    # Count the number of correct predictions
    n_correct = 0
    for i in range(len(y)):
        if y_pred[i] == y[i]:
            n_correct += 1

    print("Predicted {} correctly out of {}".format(n_correct, len(y)))
    print("Model accuracy: {}%".format(round(n_correct/len(y)*100),2))
    
print('Validation on the train set results:')
validate_clf(train_X, labels_train)

Validation on the train set results:
Predicted 4648 correctly out of 4978
Model accuracy: 93%


In [71]:
# Validate model on test set
print('Validation on the test set results:')
validate_clf(test_X, labels_test)

Validation on the test set results:
Predicted 77 correctly out of 893
Model accuracy: 9%


Try Some Different Model

In [74]:
import os
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tqdm.auto import tqdm
tqdm.pandas()

import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Dense, Input, Flatten, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

In [76]:
train = pd.read_csv('/Users/imanurrahmanemon/emon/atis_train.csv')
train.columns = ['id','intent', 'text','slots']

print(train.shape)
train.head()

(4978, 4)


,id,intent,text,slots
0,0,flight,i want to fly from boston at 838 am and arrive...,O O O O O B-fromloc.city_name O B-depart_time....
1,1,flight,what flights are available from pittsburgh to ...,O O O O O B-fromloc.city_name O B-toloc.city_n...
2,2,flight_time,what is the arrival time in san francisco for ...,O O O B-flight_time I-flight_time O B-fromloc....
3,3,airfare,cheapest airfare from tacoma to orlando,B-cost_relative O O B-fromloc.city_name O B-to...
4,4,airfare,round trip fares from pittsburgh to philadelph...,B-round_trip I-round_trip O O B-fromloc.city_n...


In [80]:
test = pd.read_csv('/Users/imanurrahmanemon/emon/atis_train.csv')
test.columns = ['id','intent', 'text','slots']

print(test.shape)
test.head()

(4978, 4)


,id,intent,text,slots
0,0,flight,i want to fly from boston at 838 am and arrive...,O O O O O B-fromloc.city_name O B-depart_time....
1,1,flight,what flights are available from pittsburgh to ...,O O O O O B-fromloc.city_name O B-toloc.city_n...
2,2,flight_time,what is the arrival time in san francisco for ...,O O O B-flight_time I-flight_time O B-fromloc....
3,3,airfare,cheapest airfare from tacoma to orlando,B-cost_relative O O B-fromloc.city_name O B-to...
4,4,airfare,round trip fares from pittsburgh to philadelph...,B-round_trip I-round_trip O O B-fromloc.city_n...


In [81]:
test.intent.value_counts(), test.intent.value_counts(normalize=True)

(intent
 flight                        3666
 airfare                        423
 ground_service                 255
 airline                        157
 abbreviation                   147
 aircraft                        81
 flight_time                     54
 quantity                        51
 flight+airfare                  21
 airport                         20
 distance                        20
 city                            19
 ground_fare                     18
 capacity                        16
 flight_no                       12
 meal                             6
 restriction                      6
 airline+flight_no                2
 ground_service+ground_fare       1
 airfare+flight_time              1
 cheapest                         1
 aircraft+flight+flight_no        1
 Name: count, dtype: int64,
 intent
 flight                        0.736440
 airfare                       0.084974
 ground_service                0.051225
 airline                       0.031539
 abb

In [82]:
train_data = train.text.values
train_labels = train.intent.values
test_data = test.text.values
test_labels = test.intent.values

len(train_data), len(train_labels), len(test_data), len(test_labels)

(4978, 4978, 4978, 4978)

In [83]:
print(train_data[123])
print(train_labels[123])

what are the nonstop flights on america west or southwest air from kansas city to burbank on saturday may twenty two
flight


In [84]:
y_train = pd.get_dummies(train_labels)
print(y_train.shape)
y_train.head()

(4978, 22)


,abbreviation,aircraft,aircraft+flight+flight_no,airfare,airfare+flight_time,airline,airline+flight_no,airport,capacity,cheapest,...,flight,flight+airfare,flight_no,flight_time,ground_fare,ground_service,ground_service+ground_fare,meal,quantity,restriction
0,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [85]:
y_test = pd.get_dummies(test_labels)
print(y_test.shape)
y_test.head()

(4978, 22)


,abbreviation,aircraft,aircraft+flight+flight_no,airfare,airfare+flight_time,airline,airline+flight_no,airport,capacity,cheapest,...,flight,flight+airfare,flight_no,flight_time,ground_fare,ground_service,ground_service+ground_fare,meal,quantity,restriction
0,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Model - Universal Sentence Encoder

In [90]:
import certifi
print(certifi.where())

/Users/imanurrahmanemon/emon/lib/python3.11/site-packages/certifi/cacert.pem


In [93]:
import ssl
print(ssl.get_default_verify_paths().openssl_cafile)

/Library/Frameworks/Python.framework/Versions/3.11/etc/openssl/cert.pem


In [91]:
import os

# Set SSL_CERT_FILE
os.environ['SSL_CERT_FILE'] = '/Users/imanurrahmanemon/emon/lib/python3.11/site-packages/certifi/cacert.pem'  # Replace with the actual path to your certificate file


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

# Manually download the model and save it locally
model_path = '/Users/imanurrahmanemon/Downloads/archive2/variables'
tf.keras.utils.get_file(model_path, module_url)

# Load the model from the local file
embed = hub.KerasLayer(model_path, trainable=True, name='USE_embedding')



In [ ]:
import os
import ssl
import tensorflow as tf
import tensorflow_hub as hub

import ssl

ssl._create_default_https_context = ssl._create_unverified_context


# Explicitly set the SSL certificate file
os.environ['SSL_CERT_FILE'] = '/Users/imanurrahmanemon/emon/lib/python3.11/site-packages/certifi/cacert.pem' # Replace with the actual path to your SSL certificate file

# Download the Universal Sentence Encoder module
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-large/4'
embed = hub.KerasLayer(module_url, trainable=True, name='USE_embedding')

# Example usage
sentences = ["what are the nonstop flights on america west or southwest air from kansas city to burbank on saturday may twenty two flight"]
embeddings = embed(sentences)

# Print the embeddings
print(embeddings)
